### <font color=black> IMPORT:: <font color=blue> **py modules** </font> </font>

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from mpl_toolkits.basemap import Basemap
import glob
import os
import shutil
import re
import cartopy
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
%matplotlib inline
#
PD=os.getcwd(); PD


'/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/2023-01_NZ_INSAR'

### <font color=black> **====================================================================** </font>
### <font color=black> Cr8:: <font color=blue> AIS FINAL DATA </font>

In [6]:
SSP='ssp585c'
PATH="/scratch/pk695/FACTS/NZrerun-pk695/local-pk695/000_full_sample_components/"
file1="NZrerun1.{arg1}.ar5AIS.ipccar5.icesheets_EAIS_localsl.nc".format(arg1=SSP)
file2="NZrerun1.{arg1}.ar5AIS.ipccar5.icesheets_WAIS_localsl.nc".format(arg1=SSP)

d_nc1=xr.open_dataset(PATH+file1)
d_nc2=xr.open_dataset(PATH+file2)

# Extract the variable to add.
slr = d_nc1['sea_level_change'] + d_nc2['sea_level_change']

# Create a copy of the old dataset (for attr/coord)
d_nc_new = d_nc1.copy(deep=True)

# Modify the variable you want to change
d_nc_new['sea_level_change'] = slr

# Save the merged dataset to a new netCDF file
d_nc_new.to_netcdf("./Devel_003_AIS/NZrerun1.{arg1}.ar5AIS.ipccar5.icesheets_AIS_localsl.nc".format(arg1=SSP))

### <font color=black> **====================================================================** </font>
### <font> :: Test Space ::
### <font color=black> **====================================================================** </font>
### <font color=black> DATA:: <font color=blue> FETCH </font>

In [2]:
path1='/scratch/pk695/FACTS/NZrerun-pk695/local-pk695/000_full_sample_components/NZrerun1.ssp119c.ar5AIS.ipccar5.icesheets_EAIS_localsl.nc'
d_nc1=xr.open_dataset(path1)
d_nc1

<xarray.Dataset>
Dimensions:           (samples: 2000, years: 14, locations: 8179)
Coordinates:
  * years             (years) int64 2020 2030 2040 2050 ... 2120 2130 2140 2150
  * locations         (locations) int64 3000000000 3000000010 ... 3000081780
  * samples           (samples) int64 0 1 2 3 4 5 ... 1995 1996 1997 1998 1999
Data variables:
    sea_level_change  (samples, years, locations) float32 ...
    lat               (locations) float64 ...
    lon               (locations) float64 ...
Attributes:
    description:  Local SLR contributions from icesheets according to AR5 wor...
    history:      Created Sun Feb 19 14:54:57 2023
    source:       SLR Framework: AR5 workflow
    scenario:     ssp119
    baseyear:     2005

In [3]:
path2='/scratch/pk695/FACTS/NZrerun-pk695/local-pk695/000_full_sample_components/NZrerun1.ssp119c.ar5AIS.ipccar5.icesheets_WAIS_localsl.nc'
d_nc2=xr.open_dataset(path1)
d_nc2

<xarray.Dataset>
Dimensions:           (samples: 2000, years: 14, locations: 8179)
Coordinates:
  * years             (years) int64 2020 2030 2040 2050 ... 2120 2130 2140 2150
  * locations         (locations) int64 3000000000 3000000010 ... 3000081780
  * samples           (samples) int64 0 1 2 3 4 5 ... 1995 1996 1997 1998 1999
Data variables:
    sea_level_change  (samples, years, locations) float32 ...
    lat               (locations) float64 ...
    lon               (locations) float64 ...
Attributes:
    description:  Local SLR contributions from icesheets according to AR5 wor...
    history:      Created Sun Feb 19 14:54:57 2023
    source:       SLR Framework: AR5 workflow
    scenario:     ssp119
    baseyear:     2005

##### <font color=black> Cr8 AIS:: <font color=blue> Combine WAIS n EAIS </font>
<p color=black> ( make sure to submit an issue on GH ) </p> 

In [4]:
# Extract the variable to add from ds2
slr1 = d_nc1['sea_level_change']
slr2 = d_nc2['sea_level_change']

# Merge the variables by adding them together
slr = slr1+slr2

# Combine the merged variables and original variables into a new dataset
#merged_ds = xr.Dataset({'sea_level_change': slr,
#                        **d_nc1.drop_vars('sea_level_change')})

# Create a copy of the dataset
d_nc_new = d_nc1.copy(deep=True)

# Modify the variable you want to change
d_nc_new['sea_level_change'] = slr

# Save the merged dataset to a new netCDF file
d_nc_new.to_netcdf('merged_file.nc')

In [5]:
d_ncF=xr.open_dataset('merged_file.nc')
d_ncF

<xarray.Dataset>
Dimensions:           (samples: 2000, years: 14, locations: 8179)
Coordinates:
  * years             (years) int64 2020 2030 2040 2050 ... 2120 2130 2140 2150
  * locations         (locations) int64 3000000000 3000000010 ... 3000081780
  * samples           (samples) int64 0 1 2 3 4 5 ... 1995 1996 1997 1998 1999
Data variables:
    sea_level_change  (samples, years, locations) float32 ...
    lat               (locations) float64 ...
    lon               (locations) float64 ...
Attributes:
    description:  Local SLR contributions from icesheets according to AR5 wor...
    history:      Created Sun Feb 19 14:54:57 2023
    source:       SLR Framework: AR5 workflow
    scenario:     ssp119
    baseyear:     2005

##### <font color=blue> Did it work :: </font>

In [12]:
i,j,k=100,13,8000
# ====
d0 = slr[i,j,k]
print('MERGED =', d0.values)
# -------
d1 = slr1[i,j,k]
print('EAIS =', d1.values)
#--
d2 = slr2[i,j,k]
print('WAIS =', d2.values)

MERGED = 2.0
EAIS = 1.0
WAIS = 1.0


In [13]:
slrF = d_ncF['sea_level_change']
dF = slrF[i,j,k]
print('MERGED =',dF.values)

MERGED = 2.0
